In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Load Oxford Pets dataset from TFDS
(train_ds, test_ds), info = tfds.load('oxford_iiit_pet:3.*.*', split=['train', 'test'], with_info=True)

# Define preprocessing functions
def preprocess_image(data):
    image = tf.image.resize(data['image'], (32, 32))  # Resize image
    image = tf.cast(image, tf.float32) / 255.0  # Normalize pixel values to [0, 1]
    label = tf.image.resize(data['segmentation_mask'], (32, 32))  # Resize label
    label = tf.cast(label, tf.int32)  # Ensure labels are integers
    
    # Convert labels to one-hot categorical
    num_classes = info.features['label'].num_classes
    label = tf.one_hot(label, depth=num_classes)
    
    # Reshape labels
    label = tf.reshape(label, (32, 32, num_classes))  # Reshape to (32, 32, num_classes)
    
    return image, label

# Apply preprocessing to datasets
train_ds = train_ds.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_ds = test_ds.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)


In [2]:
for image, label in train_ds.take(1):
    print("Image shape:", image.shape)
    print("Label shape:", label.shape)

Image shape: (32, 32, 3)
Label shape: (32, 32, 37)


In [3]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.models import Model

def create_segmentation_model(input_shape, num_classes):
    # Load ResNet50 backbone
    resnet50_base = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

    # Add segmentation-specific layers on top of ResNet50
    x = resnet50_base.output
    x = tf.keras.layers.UpSampling2D(size=(32, 32))(x)  # Upsample to match original image size
    x = tf.keras.layers.Conv2D(num_classes, kernel_size=(1, 1), activation='softmax')(x)

    # Create segmentation model
    model = tf.keras.models.Model(inputs=resnet50_base.input, outputs=x)
    return model

# Instantiate the model
input_shape = (32, 32, 3)  # Adjust input shape as per your dataset
num_classes = info.features['label'].num_classes
model = create_segmentation_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:
from keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(patience=10, monitor='loss', verbose=1)
model.fit(train_ds.batch(32), epochs=100,callbacks=[early_stopping_monitor])

Epoch 1/100
115/115 [==============================] - 19s 90ms/step - loss: 1.2544 - accuracy: 0.5335
Epoch 2/100
115/115 [==============================] - 10s 90ms/step - loss: 0.9266 - accuracy: 0.5871
Epoch 3/100
115/115 [==============================] - 11s 95ms/step - loss: 0.9036 - accuracy: 0.5988
Epoch 4/100
115/115 [==============================] - 11s 94ms/step - loss: 0.8876 - accuracy: 0.6085
Epoch 5/100
115/115 [==============================] - 11s 94ms/step - loss: 0.8686 - accuracy: 0.6169
Epoch 6/100
115/115 [==============================] - 10s 88ms/step - loss: 0.8666 - accuracy: 0.6207
Epoch 7/100
115/115 [==============================] - 11s 94ms/step - loss: 0.8610 - accuracy: 0.6263
Epoch 8/100
115/115 [==============================] - 11s 95ms/step - loss: 0.8516 - accuracy: 0.6288
Epoch 9/100
115/115 [==============================] - 10s 85ms/step - loss: 0.8452 - accuracy: 0.6308
Epoch 10/100
115/115 [==============================] - 11s 98ms/step - l

In [5]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(test_ds.batch(32))
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

115/115 [==============================] - 11s 84ms/step - loss: 0.9390 - accuracy: 0.5914
Test Loss: 0.9390224814414978
Test Accuracy: 0.591357946395874


In [6]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.models import Model
import keras

def create_segmentation_model(input_shape, num_classes):
    # Load ResNet50 backbone
    resnet50_base = keras.models.load_model(r"F:\Pre-Trained_Models\Pre-Trained_Models\oxf_32_model.h5")
    
    # Add segmentation-specific layers on top of ResNet50
    x = resnet50_base.layers[-4].output
    x = tf.keras.layers.UpSampling2D(size=(32, 32))(x)  # Upsample to match original image size
    x = tf.keras.layers.Conv2D(num_classes, kernel_size=(1, 1), activation='softmax')(x)

    # Create segmentation model
    model = tf.keras.models.Model(inputs=resnet50_base.input, outputs=x)
    return model

# Instantiate the model
input_shape = (32, 32, 3)  # Adjust input shape as per your dataset
num_classes = info.features['label'].num_classes
model = create_segmentation_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
early_stopping_monitor = EarlyStopping(patience=10, monitor='loss', verbose=1)
model.fit(train_ds.batch(32), epochs=100,callbacks=[early_stopping_monitor])

Epoch 1/100
115/115 [==============================] - 16s 89ms/step - loss: 1.0784 - accuracy: 0.5690
Epoch 2/100
115/115 [==============================] - 9s 82ms/step - loss: 0.9179 - accuracy: 0.5964
Epoch 3/100
115/115 [==============================] - 10s 91ms/step - loss: 0.9278 - accuracy: 0.5877
Epoch 4/100
115/115 [==============================] - 10s 90ms/step - loss: 0.9309 - accuracy: 0.5859
Epoch 5/100
115/115 [==============================] - 11s 89ms/step - loss: 0.9345 - accuracy: 0.5864
Epoch 6/100
115/115 [==============================] - 10s 88ms/step - loss: 0.9178 - accuracy: 0.5887
Epoch 7/100
115/115 [==============================] - 10s 90ms/step - loss: 0.9044 - accuracy: 0.5953
Epoch 8/100
115/115 [==============================] - 11s 91ms/step - loss: 0.9038 - accuracy: 0.5984
Epoch 9/100
115/115 [==============================] - 10s 88ms/step - loss: 0.9064 - accuracy: 0.5970
Epoch 10/100
115/115 [==============================] - 11s 94ms/step - lo

In [8]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(test_ds.batch(32))
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

115/115 [==============================] - 8s 62ms/step - loss: 0.9797 - accuracy: 0.5866
Test Loss: 0.9797009229660034
Test Accuracy: 0.5866425633430481


In [9]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.models import Model
import keras

def create_segmentation_model(input_shape, num_classes):
    # Load ResNet50 backbone
    resnet50_base = keras.models.load_model(r"F:\Pre-Trained_Models\Pre-Trained_Models\rnd_32_model.h5")
    
    # Add segmentation-specific layers on top of ResNet50
    x = resnet50_base.layers[-4].output
    x = tf.keras.layers.UpSampling2D(size=(32, 32))(x)  # Upsample to match original image size
    x = tf.keras.layers.Conv2D(num_classes, kernel_size=(1, 1), activation='softmax')(x)

    # Create segmentation model
    model = tf.keras.models.Model(inputs=resnet50_base.input, outputs=x)
    return model

# Instantiate the model
input_shape = (32, 32, 3)  # Adjust input shape as per your dataset
num_classes = info.features['label'].num_classes
model = create_segmentation_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
from keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(patience=10, monitor='loss', verbose=1)
model.fit(train_ds.batch(32), epochs=100,callbacks=[early_stopping_monitor])

Epoch 1/100
115/115 [==============================] - 17s 94ms/step - loss: 1.1083 - accuracy: 0.5700
Epoch 2/100
115/115 [==============================] - 10s 86ms/step - loss: 0.8959 - accuracy: 0.6016
Epoch 3/100
115/115 [==============================] - 11s 96ms/step - loss: 0.8879 - accuracy: 0.6113
Epoch 4/100
115/115 [==============================] - 11s 92ms/step - loss: 0.8825 - accuracy: 0.6198
Epoch 5/100
115/115 [==============================] - 10s 91ms/step - loss: 0.9115 - accuracy: 0.6138
Epoch 6/100
115/115 [==============================] - 11s 91ms/step - loss: 0.9273 - accuracy: 0.5876
Epoch 7/100
115/115 [==============================] - 10s 89ms/step - loss: 0.9318 - accuracy: 0.5852
Epoch 8/100
115/115 [==============================] - 10s 89ms/step - loss: 0.9087 - accuracy: 0.5868
Epoch 9/100
115/115 [==============================] - 10s 90ms/step - loss: 0.9417 - accuracy: 0.5861
Epoch 10/100
115/115 [==============================] - 10s 86ms/step - l

In [11]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(test_ds.batch(32))
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

115/115 [==============================] - 7s 55ms/step - loss: 0.9835 - accuracy: 0.5769
Test Loss: 0.9834929704666138
Test Accuracy: 0.5769003629684448
